In [0]:
#All the files (inputs.npy,target.csv, model1.h5 and model2.h5 should be in the same directory as this notebook) should be in

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import cv2
#from keras.datasets import mnist

from keras.utils import np_utils
from keras.models import Sequential,load_model
from keras.layers import Input, Conv1D, Conv2D, MaxPool2D, Flatten, Dense, Dropout, GlobalAveragePooling2D,LeakyReLU
from keras import optimizers
from sklearn.model_selection import train_test_split
import tensorflow as tf
import keras.backend as K
from keras.models import Model, Sequential
from keras.utils import plot_model
from keras.callbacks import Callback
import keras

In [0]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
import cv2 as cv
import numpy as np
import pandas as pd

In [0]:
inputs=np.load('inputs.npy')
final_input=np.zeros((inputs.shape[0],inputs.shape[1],inputs.shape[2],3))
final_input[:,:,:,0]=inputs
final_input[:,:,:,1]=inputs
final_input[:,:,:,2]=inputs

In [0]:
from numpy import loadtxt
target = loadtxt('target.csv', delimiter=',')

In [0]:
from keras import backend as K

def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [0]:
X_train,X_valid, Y_train, Y_valid = train_test_split(final_input, target[:444],test_size = 0.2, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_valid.shape,Y_valid.shape)

In [0]:
inputs_1 = Input(shape=(window_height,window_width,3),name='input_images')
x=Conv2D(32, (3, 3), activation='relu',padding='same', input_shape=(200,30))(inputs_1)
x=MaxPool2D(pool_size=(2, 2))(x)
x=Conv2D(32, (3, 3), activation='relu',padding='same')(x)
x=MaxPool2D(pool_size=(2, 2))(x)
#x=Conv2D(128, (3, 3), activation='relu',padding='same')(x)
#x=MaxPool2D(pool_size=(2, 2))(x)
#x=Conv2D(256, (3, 3), activation='relu',padding='same')(x)
#x=MaxPool2D(pool_size=(2, 2))(x)
x=Flatten()(x)
x1=Dense(64,activation='relu')(x)
x1=Dense(2,activation='softmax')(x1)

x2=Dense(64,activation='sigmoid')(x)
#x2=LeakyReLU()(x2)
x2=Dense(2,activation='sigmoid')(x2)
#x2=LeakyReLU()(x2)
model2=Model(inputs=[inputs_1],outputs=x2)
model1 = Model(inputs=[inputs_1], outputs=x1)
print(model1.summary())

In [0]:
model1.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = [f1_m,precision_m, recall_m, 'accuracy','mae'])
model2.compile(loss = 'mean_squared_error', optimizer='adam',metrics = [f1_m,precision_m, recall_m, 'accuracy','mae'])

In [0]:
model1.fit((X_train/255).astype(float),Y_train[:,:2].astype(float),batch_size=50, epochs=20, validation_data=(X_valid,Y_valid[:,:2]))

In [0]:
model2.fit((X_train/255).astype(float),Y_train[:,2:4].astype(float),batch_size=50, epochs=20, validation_data=(X_valid,Y_valid[:,2:4]))

In [0]:
model1.save('model1.h5')
model2.save('model2.h5')

In [0]:
model1=load_model('model1.h5',compile=False)
model2=load_model('model2.h5',compile=False)

In [0]:
img_number=0  # Range from 0 to 443 
pred=model1.predict(np.array([final_input[img_number]/255])), model2.predict(np.array([final_input[img_number]/255]))
if (pred[0][0,0]>0.5 or pred[0][0,1]>0.5):
  print(pred)
else:
  if(pred[0][0,0]<0.5):
    pred[1][0,0]=-1
  if(pred[0][0,1]<0.5):
    pred[1][0,1]=-1
  print(pred)
# if last two colums are -1 that means dont care since there are no respective peaks

In [0]:
img_number=0  # Range from 0 to 443 
pred=model1.predict(np.array([final_input[img_number]/255])), model2.predict(np.array([final_input[img_number]/255]))
pred

In [0]:
#for cross verification:
from numpy import loadtxt
target = loadtxt('target.csv', delimiter=',')

In [0]:
#for cross verification:
print(target)